In [ ]:
import numpy as np
from MLCode.utils import load_cup_data, np_cup, plot_NN_TR_VAL
from MLCode.NN import NN_HyperParameters
from MLCode.NN_cup import NN_Regressor, train_NN_cup
import matplotlib.pyplot as plt

X, Y = np_cup(load_cup_data())

stats = train_NN_cup(net, X_train, Y_train, X_test, Y_test, 200)
tr_error, val_error, loss = stats

plot_NN_TR_VAL(tr_error, val_error, 'MEE')
plt.show()

print('Test accuracy achieved:', max(val_accuracy))
return max(val_accuracy)